<div class='bar_title'></div>

*Introduction to Data Science (IDS)*

# Time Series Analysis

Gunther Gust <br>
Chair for Enterprise AI<br>
Data Driven Decisions (D3) Group<br>
Center for Artificial Intelligence and Data Science (CAIDAS)

<img src='https://github.com/GuntherGust/tds2_data/blob/main/images/d3.png?raw=true' style='width:20%; float:left;' />

<img src="images/CAIDASlogo.png" style="width:20%; float:left;" />

# Content (and sources)

1. [Stationarity](https://github.com/zgana/fpp3-python-readalong/blob/master/09-ARIMA-models.ipynb)
2. [Fiting an ARIMA model](https://www.geo.fu-berlin.de/en/v/soga-py/Advanced-statistics/time-series-analysis/ARIMA-modelling-in-Python/index.html) 
    * Manually
    * Auto-Arima
3. [Predicting with ARIMA models ](https://www.geo.fu-berlin.de/en/v/soga-py/Advanced-statistics/time-series-analysis/ARIMA-modelling-in-Python/index.html)
4. [Seasonal ARIMA models](https://www.kaggle.com/code/prashant111/arima-model-for-time-series-forecasting) 
5. [State-of-the-art Forecasting with Facebook's Prophet Package](https://facebook.github.io/prophet/docs/quick_start.html#python-api)
6. Related Content and further resources
    * Time series classification
    * Time Series clustering
    * Forecasting multiple time series

# Related Content and further resources

* Time series classification (Nielsen (2020) "Practical Time Series Analysis: Prediction with Statistics and Machine Learning", Chapter 9)
* Time Series clustering (Nielsen (2020), Chapter 9)
* Forecasting multiple time series (Nielsen (2020), Chapter 6)
* State Space Models (Nielsen (2020), Chapter 7)
* Overview of forecasting libraries [in Python](https://perma.cc/GEQ3-Q54X) and [R](https://perma.cc/HWY6-W2VU)
* [Twitter's Open Source Anomaly Detection Package](https://perma.cc/RV8V-PZXU)

# Introduction

Time series analysis is a branch of data science that focuses on analyzing and interpreting data points collected or recorded at __successive points in time.__ Unlike other types of data, time series data is characterized by its __temporal order:__ the time component plays a crucial role in understanding trends, patterns, and dependencies.


A time series can represent various phenomena, such as __daily stock prices, hourly weather readings, monthly sales figures, or even the number of website visits per minute.__ The key feature of time series data is that it captures how a variable evolves over time.

__Primary goals__ of Time Series Analysis are:
- Understanding trends over time
- Detecting seasonality (cycles over specific time intervals)
- Forecasting based on historical data
- Anomaly detection

### Example Task: Predicting Earth Climate

We will work with a timeseries that contains the __global annual temperature anomalies__ provided by the Berkeley Earth Surface Temperature Study. Temperatures are given in Celsius and are reported as anomalies relative to the period __January 1850 to December 2000 average.__



__Task:__ We will try to __predict the future development of the temperature__ based on the given historical data.

In [ ]:
import pandas as pd
from lets_plot import *
LetsPlot.setup_html()
import matplotlib.pyplot as plt

In [ ]:
t_global = pd.read_csv("https://raw.githubusercontent.com/vhaus63/ids_data/refs/heads/main/temp_ts.csv")
t_global["Date"] = pd.to_datetime(t_global["Date"], format="%Y-%m-%d", errors="coerce")
t_global = t_global.set_index("Date")["Monthly Anomaly_global"]

t_global

We see that the data set consist of __monthly__ mean temperatures. In order to see effects over the long period more clearly, we will aggregate the data to a __yearly basis.__

In [ ]:
temp_global_year = t_global.groupby(t_global.index.to_period("Y")).agg("mean")

There are several very helpful methods in python for dealing with timeseries. Some of those (like the above `to_period` function) we will already encounter during the course of this notebook. If you want to get further details about __Datetime manipulation,__ Resampling, Shifting, Time Zone handling,..., please refer to [this source](https://wesmckinney.com/book/time-series).

In [ ]:
temp_global_year_df = pd.DataFrame(temp_global_year).reset_index()
temp_global_year_df['Date'] = temp_global_year_df['Date'].dt.to_timestamp() 

(
    ggplot(temp_global_year_df, aes(x='Date', y='Monthly Anomaly_global'))
    + geom_line()
    + ggtitle("Earth Surface Temperature Anomalies")
    + xlab("Year") + ylab("Temperature Anomaly")
    + ggsize(1000, 400)
)

Now, we split the data set into two parts. One part, the __training set,__ is the period 1850 to 2000. Based on that data set we learn the model parameters. The other part, the __test set,__ is the period 2001 to 2021. This part is used to validate our model, as we compare the model forecast with the observations.

Obviously here, we don't want to perturbate the data points while creating training and test set since the sequentiality actually contains valuable information that we should not destroy.

In [ ]:
temp_global_training = temp_global_year["1850-01-01":"2000-01-01"]
temp_global_test = temp_global_year["2000-01-01":]

In [ ]:
training_df = temp_global_training.reset_index().rename(columns={'Date': 'year'}).assign(label='training set 1850-2000')
test_df = temp_global_test.reset_index().rename(columns={'Date': 'year'}).assign(label='test set 2001-2016')
combined_df = pd.concat([training_df, test_df])
combined_df['year'] = combined_df['year'].dt.to_timestamp()

(
    ggplot(combined_df, aes(x='year', y='Monthly Anomaly_global', color='label'))
    + geom_line()
    + ggtitle("Earth Surface Temperature Anomalies")
    + xlab("Year") + ylab("Temperature Anomaly")
    + ggsize(1000, 400)
)

It seems that there are no outliers or obviously unusual observations in the dataset.

# Stationarity


Many time series models (e.g., AR, MA, ARMA) rely on the __assumption that the series is stationary.__ If the series is not stationary, the model may not fit well or produce unreliable predictions. Additionally, stationary series are easier to forecast because their __statistical properties are stable over time.__

Thanks to the Wold decomposition theorem, any stationary data can be approximated with stationary ARMA model. Therefore one of the first steps when dealing with time series analysis is to check for stationarity.

Mathematically, stationarity requires that the statistical properties (e.g. mean, variance, and autocovariance) of the series are __time invariant.__

There are different methods for __stationarity checks:__

- __Visual inspection__
- Summary statistics:__ Calculate the mean and variance of the series over different time periods. If these statistics change significantly, the series is non-stationary.
- __Autocorrelation Plot (ACF):__ For a stationary series, the autocorrelations should decay quickly and become insignificant. For non-stationary data, the autocorrelations may show slow decay.
- __Statistical tests:__
    - Augmented Dickey-Fuller (ADF) Test
    - Kwiatkowski-Phillips-Schmidt-Shin (KPSS) Test

Based on our domain knowledge and the visualization of the time series data we are quite sure that the temperature time series is not stationary.
As a proof of concept we apply the KPSS Test.The null hypothesis in a KPSS test is that a time series is stationary.

**$H_0$: The time series is trend-stationary.**

**$H_A$: The time series is not trend-stationary.**

In [ ]:
from statsmodels.tsa.stattools import kpss

In [ ]:
def kpss_test(series, **kw):
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    # Format Output
    print(f"KPSS Statistic: {statistic}")
    print(f"p-value: {p_value}")
    print(f"num lags: {n_lags}")
    print("Critial Values:")
    for key, value in critical_values.items():
        print(f"   {key} : {value}")
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')

In [ ]:
kpss_test(temp_global_training)

The p-value of the KPSS test on the original data set (temp_global_training) is $p<0.01$. Hence we reject $H_0$ in favor of $H_A$. the original time series is __not trend-stationary.__



## Differencing

A non-stationary series can be corrected by a simple transformation such as differencing. The difference ($\delta y_t$) is calculated by subtracting one period's values from the previous period's values. Let us apply the `diff()` function and compute the KPSS test for the differenced time series.

In [ ]:
temp_global_training_diff1 = temp_global_training.diff()
kpss_test(temp_global_training_diff1.dropna())

The p-value of the KPSS test on the differenced data set (temp_global_training_diff1) is $p>0.1$. Hence we would not reject H_0 in favor of H_A; the differenced time series __is trend-stationary.__

Let's plot the differenced data set.

In [ ]:
temp_global_training_diff1_df = pd.DataFrame(temp_global_training_diff1).reset_index()
temp_global_training_diff1_df['Date'] = temp_global_training_diff1_df['Date'].dt.to_timestamp()


(
    ggplot(temp_global_training_diff1_df, aes(x='Date', y='Monthly Anomaly_global'))
    + geom_line()
    + ggtitle("Earth Surface Temperature Anomalies")
    + xlab("Year") + ylab("Temperature Anomaly")
    + ggsize(1000, 400)
)

Nice, the trend is gone visually as well. Let us plot the __autocorrelation function__ to review the correlational structure of the time series. This time we use the `plot_acf()` function from `statsmodels.graphics.tsaplots`.

## Autocorrelation function

The autocorrelation function (ACF) measures how a signal or series __correlates with itself__ at different time lags. In simpler terms, it assesses the degree to which current values of a process relate to its past values. 

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

plt.figure(figsize=(18, 6))


plot_acf(temp_global_training_diff1.dropna())
plt.title("ACF for Differenced Series")

plt.show()

## Exercise
How can we interpret the plot?

_Answer here_

In this case, we see that use the first order difference was enough to deal with the non-stationarity of our dataset. If the series were still not stationary, you could have tried to apply __second or higher-order differencing.__ Applying transformations like a __log-transformation__ can also help to make the series stationary. If it has a trend, subtracting the __trend component__ can make it stationary. If there is a seasonal pattern in the data, __seasonal differencing__ can be used to remove the seasonality.

The example plot below shows the different effects of trend and seasonality for the famous air passengers dataset:

<img src='https://raw.githubusercontent.com/vhaus63/ids_data/main/ts_decomposition.png' style='width:50%; float:left;' />


# Fitting an ARIMA model

The __ARIMA__ model stands for 

- __AR:__ Autoregression. A model that uses the dependent relationship between an observation and some number of lagged observations.
- __I:__ Integrated. The use of differencing of raw observations (e.g. subtracting an observation from an observation at the previous time step) in order to make the time series stationary.
- __MA:__ Moving Average. A model that uses the dependency between an observation and a residual error from a moving average model applied to lagged observations.



In general, an ARIMA(p,d,q) model has this equation:

\begin{equation*}
\Delta^d Y_t = \mu + \phi_1 \Delta^d Y_{t-1} + \phi_2 \Delta^d Y_{t-2} + \dots + \phi_p \Delta^d Y_{t-p} + \epsilon_t + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} + \dots + \theta_q \epsilon_{t-q}
\end{equation*}

Where:
- $ Y_t $ is the observed value at time $ t $,
- $ \Delta^d Y_t $ represents the $ d $-th differenced series (where $ d $ is the order of differencing, usually $ d = 1 $ for first-order differencing),
- $ \mu $ is the intercept (mean) term (often set to 0 for simplicity). It allows to model linear trends in the original time series.
- $ \phi_1, \phi_2, \dots, \phi_p $ are the autoregressive coefficients (AR part),
- $ \epsilon_t $ is the error term (residual) at time $ t $,
- $ \theta_1, \theta_2, \dots, \theta_q $ are the moving average coefficients (MA part),
- $ p $ is the order of the AR component (number of past values used),
- $ q $ is the order of the MA component (number of past errors used),
- $ d $ is the number of differencings applied to make the series stationary.
\end{itemize}

For example, an ARIMA(2,1,1) model has the form

\begin{equation*}
Y_t = \phi_1 Y_{t-1} + \phi_2 Y_{t-2} + \varepsilon_t + \theta_1 \varepsilon_{t-1}
\end{equation*}

with 
$Y_t = X_t - X_{t-1}$.

A timeseries that is fitted well by an ARIMA(2,1,1) model has the following characteristics:
1. Non-stationarity: The series has a trend that can be removed with first differencing.
2. Autoregressive dependence (AR(2)): The current value depends on the last two values.
3. Moving average dependence (MA(1)): The current value is influenced by the previous random shock/error.

## Steps to fit an ARIMA model

[Hyndman and Athanasopoulos (2013)](https://www.geo.fu-berlin.de/en/v/soga-py/Advanced-statistics/time-series-analysis/ARIMA-modelling-in-Python/index.html) outline the general approach for fitting an ARIMA model to a set of time series data:

1. __Plot the data.__ Identify any unusual observations.

2. If necessary, __transform__ the data (e.g. log transformations, power transformations, etc.) to stabilize the variance.

3. If the data is __non-stationary:__ take differences of the data until it becomes stationary.

4. Examine the __ACF/PACF:__ Which AR(p), MA(q), ARMA(p,q) or ARIMA(p,d,q) model is appropriate?

5. __Choose a model__ and then use the AICc as a criterion to search for a better model.

6. __Check the residuals__ from the chosen model by plotting the ACF of the residuals, and doing a Portmanteau test of the residuals. If they do not look like white noise, try a modified model.

7. __Refine__ the model using different combinations of parameters and compare the residual results and tha AICc.

8. Once you found a well fitting model, __calculate forecasts.__


The above steps fall in line with the __classical Box-Jenkins methodology.__ This process employs a meticulous blend of time series analysis and diagnostics to pinpoint the most fitting parameters.

The Box-Jenkins Methodology - A Three-Step Process:

1. _Model Identification:_ Begin with visual tools like __plots__ (e.g. ACF & PACF) and __leverage summary statistics.__ These aids help recognize trends, seasonality, and autoregressive elements. The goal here is to gauge the extent of differencing required and to determine the optimal lag size.
2. _Parameter Estimation:_ This step involves a __fitting__ procedure tailored to derive the coefficients integral to the regression model.
3. _Model Checking:_ Armed with plots and statistical tests delve into the __residual errors.__ This analysis illuminates the temporal structure that the model might have __missed.__ 

The process is repeated until either a desirable level of fit is achieved on the in-sample or out-of-sample observations (e.g. training or test datasets).

## 2. Data Transformation

The ARIMA model requires the __errors to be normally distributed.__ Data transformations are necessary if the data we are dealing with is not normal-like distributed.

The most common transformations in this case are:
- Log transformation
- Square root transformation
- Box-Cox transformation

[This article](https://medium.com/@pp1222001/types-of-function-transformations-for-better-normal-distribution-425659111b29) helps you to understand some of the methods and when to apply them.

With respect to the plot it does __not seem that the variation increases or decreases__ with the level of the time series. Thus, probably no transformation is required. However, just as a sanity check we apply a Box-Cox transformation using the `scipy.stats.boxcox` function in Python and plot the transformed data against the original data.

The Box-Cox transformation is defined as follows:

$
w_t = 
\begin{cases} 
\log(y_t), & \text{if } \lambda = 0 \\
\frac{y_t^{\lambda} - 1}{\lambda}, & \text{otherwise}
\end{cases}
$

The Box-Cox transformation depends on the parameter λ. So if λ=0 the natural logarithm is used, but if λ≠0, a power transformation is used followed by some simple scaling. We can make use of the `scipy.stats.boxcox` function, which chooses the value of λ, which __minimizes the coefficient of variation__ and uses that value within the transformation.

In [ ]:
from scipy.stats import boxcox

In [ ]:
boxcox_transformed_data, boxcox_lamba = boxcox(temp_global_training + 10) #add 10 to make all values positive
boxcox_transformed_data = pd.Series(
    boxcox_transformed_data, index=temp_global_training.index
)

We use the given λ and plot the transformed time series against the original time series.

In [ ]:
boxcox_df = pd.DataFrame(boxcox_transformed_data, columns=['temp']).reset_index()
boxcox_df['Date'] = boxcox_df['Date'].dt.to_timestamp()


(
    ggplot()
    + geom_line(data=boxcox_df, mapping=aes(x='Date', y='temp'))
    + ggsize(1000, 400)
    + ggtitle('Box-Cox transformed time series')
)

In [ ]:
training_df['year'] = training_df['year'].dt.to_timestamp()
(
    ggplot()
    + geom_line(data=training_df, mapping=aes(x='year', y='Monthly Anomaly_global'))
    + ggsize(1000, 400)
    + ggtitle('Original time series')
)

The Box-Cox transformed time series (upper plot) does not differ significantly from the original time series (lower plot). Hence, we continue our analysis with the original time series data.

## 4. Examine the ACF/PACF

To identify an appropriate time series model for a given dataset, it is essential to determine whether an autoregressive (AR) model, moving average (MA) model, or a combination of both (ARMA or ARIMA) is suitable. This process can be facilitated by analyzing the __autocorrelation function (ACF)__ and __partial autocorrelation function (PACF)__ of the series.

**ACF**: Measures the correlation of the series with its __past values (lags).__ It tells us how the current value of the series is related to its past values. The ACF is particularly useful for identifying MA processes. If it exhibits slow decay and the PACF cuts off sharply after lag p, we would identify the series as AR(p)

**PACF**: Measures the correlation between a time series and its __lagged values after removing the effects of shorter lags.__ It tells us how much a past value is related to the current value, after accounting for the influences of intermediate lags. The PACF is useful for identifying AR processes. If it shows slow decay and the ACF show a sharp cutoff after lag q, we would identify the series as being MA(q)

| Model | Description                                      | ACF Behavior            | PACF Behavior           |
|-------|--------------------------------------------------|-------------------------|-------------------------|
| AR(p) | Autoregressive model of order p                  | Slow decay              | Sharp cutoff after lag p |
| MA(q) | Moving Average model of order q                 | Sharp cutoff after lag q | Slow decay              |
| ARMA(p,q) | Combination of AR and MA models             | Slow decay              | Slow decay              |


For further reading, you can have a look at [this notebook](https://www.kaggle.com/code/iamleonie/time-series-interpreting-acf-and-pacf).

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(13, 5))

plot_acf(temp_global_training_diff1.dropna(), ax=ax[0])
ax[0].set_title("ACF")


plot_pacf(
    temp_global_training_diff1.dropna(), method="ywm", ax=ax[1]
)  ## add the calculation method running in the background ("ywm")

ax[1].set_title("PACF")
plt.show()



It seems that the ACF behaves like a damped sine wave and that the PACF __cuts off after lag p=3.__ One good starting point would be an __ARIMA(3, 1, 0)__ process for the `temp_global_training` time series.

### Exercise
Why do we use the "raw" and not the already differenciated timeseries `temp_global_training` for the ARIMA model but inspected the differenciated timeseries `temp_global_training_diff1`? Could we also use the `temp_global_training_diff1` for the model? If so, which approach is better and why?

_Answer here_

## 5. Model Selection

Our examination of ACF and PACF suggested to start with a ARIMA(3,1,0) model. Let's see how this performs.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
model = ARIMA(temp_global_training, order=(3, 1, 0))
model_fit = model.fit()
print(model_fit.summary())

What does the second block in the results above mean?
Remember that the AR-part of an ARIMA model deals with how to explain the current value based on a linear combination of a certain (q=3) number of past values. That means that the coefficients you see here are the coefficients of the linear combination:

$-0.5353y_{t-1}-0.3974y_{t-2}-0.3856y_{t-3}+w_t.$

Since the MA-part is set to 0 in our case, its parameters don't appear in the model. The term $w_t$ comes from adding an error term that describes the white noise with a standard deviation of $\sqrt\sigma =\sqrt{0.038} = 0.195.$

Because we are using a first-order differenciated model (d=1), the correct equation to describe it is then

$\Delta y_t = -0.5353y_{t-1}-0.3974y_{t-2}-0.3856y_{t-3}+w_t.$

We want to find out how well our model performs. From previous lectures, we are already familiar with the AIC (Akaike Information Criterion). Since in this specific case, we have only relatively few data points (151) compared to the model parameters that we want to estimate (4), we will use a slight adaption of the AIC, the AICc:

$AICc=AIC+\frac{2k(k+1)}{n-k-1}$

Where:
- k is the number of estimated parameters and
- n is the sample size.

The additional term penalizes models more heavily when the sample size is small, preventing overfitting due to the overestimation of parameters. As n grows to infinity, AICc converges to AIC. A rule of thumb is to use the AICc as long as  $\frac{\text{number of data points}}{\text{number of parameters}} = \frac{n}{k}< 40.$

In [ ]:
model_fit.aicc

In ARIMA modelling it is good practice to check some more models and compare them by comparing the AICc.

### Exercise
Fit some variations of ARIMA(p, d, q) models including ARIMA(3, 1, 1), ARIMA(3, 1, 2), ARIMA(2, 1, 2) and compare their AICc. Which model should be chosen?

In [ ]:
### Your code here

## Auto ARIMA

The `pmdarima` (pyramid-arima) package includes the `auto.arima()` function. The function uses a __variation of the Hyndman and Khandakar algorithm__ which combines unit root tests, minimization of the AICc and maximum likelihood estimation (MLE) to obtain an optimized ARIMA model. In short terms, it generates a set of optimal (p, d, q) that optimizes model fit criteria by searching through combinations of order parameters. See the documentation of the function [here](https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.auto_arima.html).

In [ ]:
from pmdarima.arima import auto_arima

In [ ]:
temp_global_training.index = temp_global_training.index.to_timestamp()
auto_model = auto_arima(temp_global_training, information_criterion='aicc', trace=True)
auto_model.summary()

In [ ]:
auto_model.aicc()

The `auto.arima()` function returns an ARIMA(2, 1, 3) process, with an AICc of -66.839 This metric is even lower compared to our best model so far. Note here that the difference in the printed AICc values comes from the function using an __approximation__ during the testing of the different models in order to speed up the process.

## 6. Check the residuals

The _residuals_ in a time series model are what is left over after fitting a model. For many (but not all) time series models, the residuals are equal to the difference between the observations and the corresponding fitted values: $e_t=y_t−\hat{y}_t.$

Residuals are useful in checking whether a model has adequately captured the information in the data. A good forecasting method will yield residuals with the following properties:

1. The residuals are __uncorrelated.__ If there are correlations between residuals, then there is information left in the residuals which should be used in computing forecasts.
2. The residuals have __zero mean.__ If the residuals have a mean other than zero, then the forecasts are biased.

Any forecasting method that does not satisfy these properties can be improved. However, that does not mean that forecasting methods that satisfy these properties cannot be improved. It is possible to have several different forecasting methods for the same data set, all of which satisfy these properties. Checking these properties is important in order to see whether a method is using all of the available information, but it is not a good way to select a forecasting method.

Let's examine the ACF of the residuals from the `auto_model` model.

In [ ]:
residuals = pd.DataFrame(auto_model.resid(), columns=['Residuals']).reset_index()
residuals

In [ ]:
auto_model.plot_diagnostics(figsize=(10,8))
plt.show()

_Standardized residual:_ The residual errors seem to fluctuate around a __mean of zero__ and have a __uniform variance.__

_Histogram:_ The density plot suggest __normal distribution.__

_Theoretical Quantiles:_ Mostly the dots __fall perfectly in line__ with the red line. Any significant deviations would imply the distribution is skewed.

_Correlogram:_ The Correlogram, (or ACF plot) shows the residual errors are __not autocorrelated.__ The ACF plot shows all correlations within the threshold limits (blue band) indicating that the residuals are behaving like white noise.

Overall, the model seems to be a __good fit.__ So, let's use it to forecast.

## 8. Calculate Forecasts

Forecasting using a fitted model is straightforward in Python. We use the `forecast()` function from the statsmodels package. We specify forecast horizon in steps, the number of out of sample forecasts from the end of the sample and use the fitted model to generate those predictions. In addition we specify $\alpha$, to display __confidence intervals__ for the forecasts $(1 - \alpha \%)$.

In [ ]:
model = ARIMA(temp_global_training, order=(2,1,3), trend="t") #use the model from auto.arima that includes an intercept
fitted = model.fit()
forecast_series = fitted.forecast(40, alpha=0.05)

In [ ]:
df_fitted = pd.DataFrame(fitted.fittedvalues, columns=['Fitted']).reset_index()
(
    ggplot() +
    geom_line(data=training_df, mapping=aes(x='year', y='Monthly Anomaly_global'), color='black', size=.7, manual_key='Training Set 1850-2000') +
    geom_line(data=df_fitted, mapping=aes(x='Date', y='Fitted'), color='blue', size=.7, manual_key='Fitted Data') +
    ggtitle('Earth Surface Temperature Anomalies') +
    xlab('Year') +
    ylab('Temperature Anomalies') +
    ggsize(1000, 400)
)

For each time \(t\) in your historical training data (from the earliest time to the last observed time), the __fitted value $\hat{Y}_t$__ is typically the one-step-ahead forecast using data up to time \(t-1\). In other words:

$\hat{Y}_t = E\bigl(Y_t \mid Y_{t-1}, Y_{t-2}, \dots \bigr)$

given the estimated model parameters.


### Plot the forecast (and confidence intervals)

In [ ]:
forecast = fitted.get_forecast(40)
conf_int_80 = forecast.conf_int(alpha=0.2)
conf_int_95 = forecast.conf_int(alpha=0.05)

In [ ]:
conf_int_95.reset_index(names='Date', inplace=True)
conf_int_80.reset_index(names='Date', inplace=True)
forecast_series = pd.DataFrame(forecast_series).reset_index(names='Date')

In [ ]:
(
    ggplot() +
    geom_line(data=combined_df, mapping=aes(x='year', y='Monthly Anomaly_global', color='label')) +
    geom_line(
        data=forecast_series,
        mapping=aes(x='Date', y='predicted_mean'),
        color='blue',
        size=0.7,
        manual_key='Forecast'
    ) +
    geom_ribbon(
        data=conf_int_95,
        mapping=aes(
            x='Date',
            ymin='lower Monthly Anomaly_global',
            ymax='upper Monthly Anomaly_global'
        ),
        fill='blue',
        alpha=0.1,
        manual_key='95% Confidence Interval'
    ) +
    geom_ribbon(
        data=conf_int_80,
        mapping=aes(
            x='Date',
            ymin='lower Monthly Anomaly_global',
            ymax='upper Monthly Anomaly_global'
        ),
        fill='blue',
        alpha=0.2,
        manual_key='80% Confidence Interval'
    ) +
    ggtitle('Earth Surface Temperature Anomalies') +
    xlab('Year') +
    ylab('Temperature Anomalies') +
    ggsize(1000, 400)

)

# Seasonal ARIMA models (SARIMA)

The plain ARIMA model has a problem: it does not support seasonality. If the time series has defined seasonality, then we should go for Seasonal ARIMA model (in short SARIMA) which uses seasonal differencing.

The earth surface temperature data does not contain any obvious seasonality pattern, but what if we have to deal with seasonal data?

For example, look at the following data on Drug Sales:

In [ ]:
drug_data = pd.read_csv('https://raw.githubusercontent.com/vhaus63/ids_data/refs/heads/main/drug_sales.txt', sep=",", header=0, parse_dates=['date'], index_col='date')
drug_df = drug_data.reset_index()

(
    ggplot(drug_df, aes(x='date', y='value'))
    + geom_line()
    + ggtitle("Drug Sales")
    + ggsize(1000, 400)
)

SARIMA is particularly useful for forecasting when data exhibit regular, predictable changes that recur every calendar cycle, such as __daily, monthly, or quarterly fluctuations (in short, seasonality).__ The model is capable of capturing both the trends and seasonality in data to make accurate predictions.

A SARIMA model is represented as

__ARIMA (p,d,q) (P,D,Q)s,__

where the second part is the seasonal part of the model:
1. s is the length of the season
2. Seasonal Autoregressive (P): This component captures the relationship between the current value of the series and its past values, specifically at seasonal lags.
3. Seasonal Integrated (D): Similar to the non-seasonal differencing, this component accounts for the differencing required to remove seasonality from the series.
4. Seasonal Moving Average (Q): This component models the dependency between the current value and the residual errors of the previous predictions at seasonal lags.

Let's look at an example:
The SARIMA model $(1, 0, 0)(1, 1, 1) S$ can be expressed as:

\begin{equation*}
y''_t = \phi_1 y''_{t-1} + \Phi_S y''_{t-S} + \epsilon_t + \Theta_S \epsilon_{t-S}
\end{equation*}

Where:
- $y''_t = y_t - y_{t-S}$: Seasonally differenced series ($D = 1$).
- $\phi_1$: Coefficient of the non-seasonal AR(1) term.
- $\Phi_S$: Coefficient of the seasonal AR(1)
- $\Theta_S$: Coefficient of the Seasonal MA(1)


## Seasonal Differencing

Seasonal data is not stationary by design. If we apply differencing, the data (can) become stationary. However that does not mean that the differencing removes all the seasonal correlation: What happened in one seasons' datapoints can still impact the current seasons' datapoints. This is what the seasonality part of the SARIMA model deals with.

### a) Regular Differencing

In [ ]:
df = pd.DataFrame({
    'index': drug_df['date'],
    'original': drug_df['value'],
    'diff_1': drug_df['value'].diff(1),
    'diff_12': drug_df['value'].diff(12)
})

# Usual Differencing Plot
plot1 = ggplot(df) + \
    geom_line(aes(x='index', y='original'), color='blue', size=1, label='Original Series') + \
    geom_line(aes(x='index', y='diff_1'), color='red', size=1, label='Usual Differencing') + \
    labs(title='Usual Differencing', x='Time', y='Value') + \
    theme(legend_position='top') + \
    scale_color_manual(values=['blue', 'red']) + \
    ggsize(1000, 400)
    
plot1.show()


### b) Seasonal Differencing

In [ ]:
# Seasonal Differencing Plot
plot2 = ggplot(df) + \
    geom_line(aes(x='index', y='original'), color='blue', size=1, label='Original Series') + \
    geom_line(aes(x='index', y='diff_12'), color='green', size=1, label='Seasonal Differencing') + \
    labs(title='Seasonal Differencing', x='Time', y='Value') + \
    theme(legend_position='top') + \
    scale_color_manual(values=['blue', 'green']) +\
    ggsize(1000, 400)

plot2.show()

We can see that, the seasonal spikes are intact after applying usual differencing (lag 1). Whereas, it is rectified after seasonal differencing. Still, we have to assume even with the now stationary data that seasonal trends influcence future values. Since this relationship is not captured by an ARIMA model, we will try to use SARIMA now.

## Model estimation using auto_arima()

Let’s build the SARIMA model using pmdarima‘s auto_arima(). To do so, we need to set seasonal=True, set the frequency __m=12 for month wise series and enforce D=1.__ (You do not have to fix this. However, the seasonal identification doesn’t always happen or may not be correct for every use case, so it’s generally safer to set m and D explicitly if you know the seasonal cycle.)

In [ ]:
smodel = auto_arima(drug_data, start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3, m=12,
                         start_P=0, seasonal=True,
                         d=None, D=1, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

smodel.summary()

The model has estimated the AIC and the P values of the coefficients look significant.

Let's look at the residual diagnostics plot.

In [ ]:
smodel.plot_diagnostics(figsize=(10,8))
plt.show()

The residual plot reveals an increasing variance. The large residual values are also visiable in QQplot. We could try to handle this using a statistical transformation (e.g. Box-Cox transform) to stabilize the residuals. However, we omit this here. 

Let's forecast the next 24 months.

In [ ]:
n_periods = 24
fitted, confint = smodel.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = pd.date_range(drug_data.index[-1], periods = n_periods, freq='MS')

In [ ]:
fitted_series = pd.Series(fitted, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

forecast_data = pd.DataFrame({
    'Date': index_of_fc,
    'Fitted': fitted_series,
    'Lower': lower_series,
    'Upper': upper_series
})

p = ggplot() + \
    geom_line(data=drug_df, mapping=aes(x='date', y='value'), color='blue') + \
    geom_line(data=forecast_data, mapping=aes(x='Date', y='Fitted'), color='darkgreen') + \
    geom_ribbon(data=forecast_data, mapping=aes(x='Date', ymin='Lower', ymax='Upper'), fill='black', alpha=0.15) + \
    labs(title="SARIMA - Final Forecast of Drug Sales - Time Series Dataset") + \
    ggsize(1000,400)

p.show()

There we have a nice forecast that captures the expected seasonal demand pattern.

# Prophet: Facebooks forecasting package

Prophet is an open-source tool released by Facebook's Data Science team that produces time series forecasting data based on an additive model where a __non-linear trend__ fits with __seasonality__ and __holiday effects.__ The design principles allow parameter adjustments without much knowledge of the underlying model which makes the method applicable to teams with less statistical knowledge.

As an example, let’s look at a time series of the log daily __page views for the Wikipedia page for Peyton Manning__ (a famous American football quarterback). Peyton Manning provides a nice example because it illustrates some of Prophet’s features, like multiple seasonality, changing growth rates, and the ability to model special days (such as Manning’s playoff and superbowl appearances).

In [ ]:
from prophet import Prophet

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')

(
    ggplot(df, aes(x='ds', y='y'))
    + geom_line()
    + ggtitle("Page views for Peyton Manning")
    + xlab("Year") + ylab("Views")
    + ggsize(1000, 400)
)

We fit the model by instantiating a new `Prophet` object and then fit it on the historical data.

In [ ]:
m = Prophet()
m.fit(df)

In order to make predictions, we call the method `make_future_dataframe`. It creates a dataframe where we can store the predicted values.

In [ ]:
future = m.make_future_dataframe(periods=365)

Now we predict into the specified future and look at the results. 

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

Here, we can examine the different components our timeseries was made of. The plots show by default __the trend, yearly seasonality, and weekly seasonality.__ Prophet decomposes the timeseries already very well without any further specifications.

# Related Content and further resources

## Time Series Classification
<img src='https://raw.githubusercontent.com/vhaus63/ids_data/main/ts_classification.png' style='width:40%; float:left;' />


The goal of this supervised learning task is to assign a specific __label or category to each time series__ based on its content or pattern. The classification is done according to predefined classes or categories. The model learns to predict the label of new, unseen time series based on patterns learned from the training data.

This plays an important role in __classification of sensor data__ to predict if a machine is operating normally or has a fault or identifying activities such as running, swimming, sitting from wearable devices.

One of the most popular and traditional Time Series Classification approaches is the use of a nearest neighbor (NN) classifier coupled with a distance function. Particularly, the Dynamic Time Warping (DTW) distance when used with a NN classifier has been shown to be a very strong algorithm. Additionally to this, research has found different Deep Neural Network Architectures to be very performant in this task.


## Time Series Clustering
<img src='https://raw.githubusercontent.com/vhaus63/ids_data/main/clustering.png' style='width:70%; float:left;' />

Time Series Clustering on the other hand aims at __grouping similar time series into clusters__ without prior knowledge of the categories or labels. The data points are grouped based on their patterns or features. It is an unsupervised learning task.

Usecases might be:
- Detecting patterns of similar behaviors in user activity over time,
- Grouping sensor data from different machines in a factory into clusters based on similar operational patterns.

## Forecasting Multiple Time Series

In the real world, we are often lucky enough to have __several time series in parallel__ that
are presumably related to one another. __Vector Autoregression (VAR)__ is a multivariate forecasting algorithm that is used when two or more time series influence each other.

Ok, so how is VAR different from other Autoregressive models like AR, ARMA or ARIMA?

The primary difference is those models are uni-directional, where, the predictors influence the Y and not vice-versa. Whereas, __Vector Auto Regression (VAR) is bi-directional.__ That is, the variables influence each other. Therefore, each variable is modeled as a linear combination of past values of itself and the past values of other variables in the system. Since you have multiple time series that influence each other, it is modeled as a system of equations with one equation per variable (time series). That is, if you have 5 time series that influence each other, we will have a system of 5 equations.

For further reading please refer to [this](https://github.com/xxl4tomxu98/vector-autoregressive-model-wage-inflations/blob/main/VAR-model.ipynb) notebook.

## Outlook

This lecture focused on forecasting time series data using the ARIMA model. However, it is important to note that there are many other approaches for forecasting tasks that we did not cover here. These range from traditional statistical models like Exponential Smoothing or State Space Models to Machine Learning models such as Random Forests or Support Vector Machines, and even advanced Deep Learning models. Choosing the most appropriate model always depends on the specific task and the characteristics of the data.

Beyond forecasting, time series analysis encompasses other exciting areas, such as Anomaly Detection and pattern recognition, offering diverse tools and methods to tackle real-world challenges. This lecture serves as a solid foundation for further exploration into these advanced topics.

## Mentimeter

<img src='images/d3.png' style='width:80%; float:left;' />